## Importing all the necessary dependencies

In [1]:
import numpy as np 
import pandas as pd 
import ast
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## Reading the CSV files

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
1751,27000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",NaN,9583,"[{""id"": 2908, ""name"": ""secret society""}, {""id""...",en,Divine Secrets of the Ya-Ya Sisterhood,A mother and daughter dispute is resolved by t...,5.576937,"[{""name"": ""All Girl Productions"", ""id"": 10535}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2002-06-07,0,116.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Secret Is Out.,Divine Secrets of the Ya-Ya Sisterhood,5.6,65
56,185000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.startrekmovie.com/,188927,"[{""id"": 9663, ""name"": ""sequel""}, {""id"": 9743, ...",en,Star Trek Beyond,The USS Enterprise crew explores the furthest ...,65.352913,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2016-07-07,343471816,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Star Trek Beyond,6.6,2568
2357,18000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",NaN,38665,"[{""id"": 178867, ""name"": ""lone ranger""}, {""id"":...",en,The Legend of the Lone Ranger,"When the young Texas Ranger, John Reid, is the...",11.725805,"[{""name"": ""ITC Entertainment"", ""id"": 1584}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1981-08-20,2945600,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"The untold story of the man behind the mask, a...",The Legend of the Lone Ranger,3.7,14
1563,30000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,10333,"[{""id"": 579, ""name"": ""american football""}, {""i...",en,The Prince of Tides,A troubled man talks to his suicidal sister's ...,3.845231,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1991-12-25,74787599,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"A story about the memories that haunt us, and ...",The Prince of Tides,5.9,56
1218,40000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,82687,"[{""id"": 1436, ""name"": ""inventor""}, {""id"": 7312...",en,The Guilt Trip,An inventor and his mom hit the road together ...,15.283444,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-12-19,41863726,95.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Get ready for one mother of a road trip,The Guilt Trip,5.4,284


In [4]:
credits.sample(5)

,movie_id,title,cast,crew
709,38167,Eat Pray Love,"[{""cast_id"": 4, ""character"": ""Elizabeth Gilber...","[{""credit_id"": ""52fe469c9251416c91059ee1"", ""de..."
4194,1999,In the Bedroom,"[{""cast_id"": 13, ""character"": ""Matt Fowler"", ""...","[{""credit_id"": ""52fe432ac3a36847f803f5e7"", ""de..."
4326,94329,The Raid,"[{""cast_id"": 11, ""character"": ""Rama"", ""credit_...","[{""credit_id"": ""52fe49529251416c750c441d"", ""de..."
3479,1443,The Virgin Suicides,"[{""cast_id"": 17, ""character"": ""Mr. Lisbon"", ""c...","[{""credit_id"": ""52fe42f8c3a36847f80308d5"", ""de..."
4563,15239,The Toxic Avenger,"[{""cast_id"": 1, ""character"": ""The Toxic Avenge...","[{""credit_id"": ""53f7425ac3a36833f4005420"", ""de..."


In [5]:
movies = movies.merge(credits, on='title') 

In [6]:
movies.sample(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
2019,22000000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 18, ""na...",NaN,251,"[{""id"": 417, ""name"": ""corruption""}, {""id"": 663...",en,Ghost,"Sam Wheat is a banker, Molly Jensen is an arti...",41.967005,"[{""name"": ""Paramount Pictures"", ""id"": 4}]",...,127.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A love that will last forever.,Ghost,6.9,1339,251,"[{""cast_id"": 2, ""character"": ""Sam Wheat"", ""cre...","[{""credit_id"": ""52fe422dc3a36847f8009f89"", ""de..."
4758,65000,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 35, ...",http://tinyfurniture.com/,47607,"[{""id"": 1156, ""name"": ""sister sister relations...",en,Tiny Furniture,"After graduating from film school, Aura return...",2.380332,"[{""name"": ""Tiny Ponies"", ""id"": 65873}]",...,99.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Aura would like you to know that she is having...,Tiny Furniture,5.6,59,47607,"[{""cast_id"": 12, ""character"": ""Aura"", ""credit_...","[{""credit_id"": ""52fe473bc3a36847f812a649"", ""de..."
2496,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 36, ""name...",NaN,18840,[],en,The White Countess,The last movie from the team of Ismail Merchan...,4.488547,"[{""name"": ""Sony Pictures Classics"", ""id"": 58},...",...,135.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,NaN,The White Countess,6.9,21,18840,"[{""cast_id"": 3, ""character"": ""Todd Jackson"", ""...","[{""credit_id"": ""59bfa1c19251414759012fd9"", ""de..."
2641,12000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://www.banlieue13-lefilm.com/,10045,"[{""id"": 90, ""name"": ""paris""}, {""id"": 258, ""nam...",fr,Banlieue 13,"Set in the ghettos of Paris in 2010, an underc...",17.493735,"[{""name"": ""Canal Plus"", ""id"": 104}, {""name"": ""...",...,84.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}]",Released,Welcome to District B13. Welcome to the future,District B13,6.5,565,10045,"[{""cast_id"": 19, ""character"": ""Le\u00efto"", ""c...","[{""credit_id"": ""52fe43119251416c750023b3"", ""de..."
1128,42000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 9648, ""na...",NaN,11560,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,High Crimes,High powered lawyer Claire Kubik finds her wor...,14.046705,"[{""name"": ""Twentieth Century Fox Film Corporat...",...,115.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Everything you trust. Everything you know. May...,High Crimes,6.1,184,11560,"[{""cast_id"": 1, ""character"": ""Claire Chapman"",...","[{""credit_id"": ""597aa90cc3a368374c004c53"", ""de..."


In [7]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [8]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2693,10368,Tea with Mussolini,Semi-autobiographical film directed by Franco ...,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 131, ""name"": ""italy""}, {""id"": 612, ""na...","[{""cast_id"": 1, ""character"": ""Elsa Armistan"", ...","[{""credit_id"": ""52fe43619251416c7500e239"", ""de..."
1652,10350,Wing Commander,The Hollywood version of the popular video gam...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 878, ""na...","[{""id"": 1721, ""name"": ""fight""}, {""id"": 3203, ""...","[{""cast_id"": 1, ""character"": ""1st Lt. Christop...","[{""credit_id"": ""52fe435e9251416c7500db63"", ""de..."
2047,1599,I Heart Huckabees,"A husband-and-wife team play detective, but no...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...","[{""id"": 572, ""name"": ""sex""}, {""id"": 703, ""name...","[{""cast_id"": 1, ""character"": ""Albert Markovski...","[{""credit_id"": ""52fe4303c3a36847f8033e31"", ""de..."
1475,2900,The Astronaut's Wife,When astronaut Spencer Armacost returns to Ear...,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 878, ""nam...","[{""id"": 1157, ""name"": ""wife husband relationsh...","[{""cast_id"": 16, ""character"": ""Commander Spenc...","[{""credit_id"": ""52fe4373c3a36847f805529f"", ""de..."
3093,14195,Cecil B. Demented,An insane independent film director and his re...,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 35, ""n...","[{""id"": 2671, ""name"": ""heroin""}, {""id"": 9782, ...","[{""cast_id"": 1, ""character"": ""Honey Whitlock"",...","[{""credit_id"": ""562264eec3a368240d004758"", ""de..."


## Data Preprocessing

In [9]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [10]:
movies.dropna(inplace=True)

In [11]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [12]:
movies.duplicated().sum()

0

In [13]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [14]:
def create_genre_list_of_movie(movie_genre_object_array):
    
    genre_name_list = []
    
    for i in ast.literal_eval(movie_genre_object_array):
        
        
        genre_name_list.append(i['name'])
        
        
    return genre_name_list

In [15]:
movies['genres'] = movies['genres'].apply(create_genre_list_of_movie)

In [16]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
721,10628,Kangaroo Jack,"Two childhood friends, a New York hairstylist ...","[Comedy, Adventure, Crime]","[{""id"": 1322, ""name"": ""money delivery""}, {""id""...","[{""cast_id"": 1, ""character"": ""Charlie Carbone""...","[{""credit_id"": ""52fe43989251416c750167f3"", ""de..."
3680,77949,The Awakening,1921 England is overwhelmed by the loss and gr...,"[Horror, Thriller]","[{""id"": 236, ""name"": ""suicide""}, {""id"": 392, ""...","[{""cast_id"": 1021, ""character"": ""Florence Cath...","[{""credit_id"": ""54e6fd789251416f58000ee8"", ""de..."
2208,12797,Ghost Town,"Dentist, Bertram Pincus is a man whose people ...","[Comedy, Fantasy, Romance]","[{""id"": 3737, ""name"": ""dying and death""}, {""id...","[{""cast_id"": 1, ""character"": ""Bertram Pincus"",...","[{""credit_id"": ""52fe45249251416c7504d1bb"", ""de..."
3514,837,Videodrome,A sleazy cable-TV programmer begins to see his...,"[Horror, Mystery, Science Fiction]","[{""id"": 236, ""name"": ""suicide""}, {""id"": 2571, ...","[{""cast_id"": 13, ""character"": ""Max Renn"", ""cre...","[{""credit_id"": ""52fe427ec3a36847f80231e1"", ""de..."
2533,17202,Why Did I Get Married?,The film is about the difficulty of maintainin...,"[Comedy, Drama]",[],"[{""cast_id"": 2, ""character"": ""Terry"", ""credit_...","[{""credit_id"": ""52fe470f9251416c7508c81d"", ""de..."


In [17]:
def create_keyword_list_of_movie(movie_keyword_object_array):
    
    keyword_name_list = []
    
    for i in ast.literal_eval(movie_keyword_object_array):
        
        keyword_name_list.append(i['name'])
        
        
    return keyword_name_list

In [18]:
movies['keywords'] = movies['keywords'].apply(create_keyword_list_of_movie)

In [19]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
912,628,Interview with the Vampire,"A vampire relates his epic life story of love,...","[Horror, Romance]","[paris, san francisco, vampire, plantation, pi...","[{""cast_id"": 18, ""character"": ""Louis de Pointe...","[{""credit_id"": ""52fe4260c3a36847f80199bf"", ""de..."
3601,14745,The Haunting of Molly Hartley,When teenage Molly Hartley moves to a new town...,"[Drama, Horror]",[],"[{""cast_id"": 1, ""character"": ""Molly Hartley"", ...","[{""credit_id"": ""52fe46159251416c7506c805"", ""de..."
309,10214,Son of the Mask,"Tim Avery, an aspiring cartoonist, finds himse...","[Fantasy, Comedy, Family, Adventure]","[baby, mask, viking]","[{""cast_id"": 1, ""character"": ""Tim Avery"", ""cre...","[{""credit_id"": ""52fe43439251416c75009b91"", ""de..."
96,27205,Inception,"Cobb, a skilled thief who commits corporate es...","[Action, Thriller, Science Fiction, Mystery, A...","[loss of lover, dream, kidnapping, sleep, subc...","[{""cast_id"": 1, ""character"": ""Dom Cobb"", ""cred...","[{""credit_id"": ""56e8462cc3a368408400354c"", ""de..."
4638,50035,Lovely & Amazing,Self-esteem and insecurity are at the heart of...,"[Comedy, Drama, Romance]",[woman director],"[{""cast_id"": 1, ""character"": ""Michelle Marks"",...","[{""credit_id"": ""52fe47b6c3a36847f8144399"", ""de..."


In [20]:
def create_top_three_cast_list_of_movie(cast_object_array):
    
    top_three_cast_name_list = []
    
    counter = 0
    
    for i in ast.literal_eval(cast_object_array):
        
        if counter != 3:
            
            top_three_cast_name_list.append(i['name'])
            
            counter += 1
        
        else:
            
            break

        
    return top_three_cast_name_list

In [21]:
movies['cast'] = movies['cast'].apply(create_top_three_cast_list_of_movie)

In [22]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
4689,5759,Baghead,Four actors go to a cabin in the woods to writ...,"[Drama, Comedy, Horror]","[mumblecore, mumblegore]","[Ross Partridge, Steve Zissis, Greta Gerwig]","[{""credit_id"": ""52fe4420c3a36847f8083bb5"", ""de..."
2609,115,The Big Lebowski,"Jeffrey ""The Dude"" Lebowski, a Los Angeles sla...","[Comedy, Crime]","[white russian, dude, bowling, vietnam veteran...","[Jeff Bridges, John Goodman, Julianne Moore]","[{""credit_id"": ""52fe4219c3a36847f8004109"", ""de..."
3522,89861,Stiff Upper Lips,Stiff Upper Lips is a broad parody of British ...,[Comedy],"[italy, victorian england, young woman]","[Georgina Cates, Peter Ustinov, Prunella Scales]","[{""credit_id"": ""52fe4a2f9251416c910c5edb"", ""de..."
3639,36351,Light Sleeper,A drug dealer with upscale clientele is having...,"[Drama, Crime]","[suicide, drug dealer, redemption, addict, exi...","[Willem Dafoe, Susan Sarandon, Dana Delany]","[{""credit_id"": ""52fe45e19251416c91041985"", ""de..."
3436,87499,The East,An operative for an elite private intelligence...,"[Drama, Thriller]","[secret organization, murder, environmentalism...","[Brit Marling, Alexander Skarsgård, Ellen Page]","[{""credit_id"": ""52fe49ca9251416c910b8cbf"", ""de..."


In [23]:
def crew_list_of_movie(crew_object_array):
    
    crew_name_list = []
    
    for i in ast.literal_eval(crew_object_array):
        
        if i['job'] == 'Director':
            
            crew_name_list.append(i['name'])
            
            break
            
            # since every row represents a movie and every movie has only 1 director, therefore, as soon as we get the 
            # name of the director, we break out of the loop
            
    return crew_name_list

In [24]:
movies['crew'] = movies['crew'].apply(crew_list_of_movie)

In [25]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
4078,11561,Sleeper,"Miles Monroe, a clarinet-playing health food s...","[Comedy, Romance, Science Fiction]","[sex, revolution, future, dystopia, government...","[Woody Allen, Diane Keaton, John Beck]",[Woody Allen]
1165,4234,Scream 3,"A murdering spree begins to happen again, this...","[Horror, Mystery]","[mask, metal detector, film director, ex-cop, ...","[David Arquette, Neve Campbell, Courteney Cox]",[Wes Craven]
177,77950,Turbo,The tale of an ordinary garden snail who dream...,"[Animation, Family]","[underdog, car race, dream, speed, power, snai...","[Ryan Reynolds, Paul Giamatti, Michael Peña]",[David Soren]
2457,639,When Harry Met Sally...,"During their travels from Chicago to New York,...","[Comedy, Romance, Drama]","[new york, wife husband relationship, restaura...","[Meg Ryan, Billy Crystal, Carrie Fisher]",[Rob Reiner]
3214,25196,Crazy Heart,When reporter Jean Craddock interviews Bad Bla...,"[Drama, Music, Romance]","[taxi, country music, journalist, guitar, bar,...","[Jeff Bridges, Maggie Gyllenhaal, Colin Farrell]",[Scott Cooper]


In [26]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [27]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
2641,10045,District B13,"[Set, in, the, ghettos, of, Paris, in, 2010,, ...","[Action, Thriller, Science Fiction]","[paris, bomb, vororte, dystopia, parkour, gang...","[David Belle, Cyril Raffaelli, Tony D'Amario]",[Pierre Morel]
2560,64690,Drive,"[A, Hollywood, stunt, performer, who, moonligh...","[Drama, Action, Thriller, Crime]","[stuntman, blood splatter, independent film, p...","[Ryan Gosling, Carey Mulligan, Bryan Cranston]",[Nicolas Winding Refn]
1516,12106,The Quick and the Dead,"[A, mysterious, woman, comes, to, compete, in,...","[Action, Western]","[gunslinger, revenge, prairie, shootout, pistol]","[Sharon Stone, Gene Hackman, Russell Crowe]",[Sam Raimi]
1968,11635,Old School,"[Three, friends, attempt, to, recapture, their...",[Comedy],"[sex doll, streaking, mouth to mouth resuscita...","[Luke Wilson, Will Ferrell, Vince Vaughn]",[Todd Phillips]
4547,21801,The Slaughter Rule,"[A, young, man, finds, solace, with, a, young,...",[Drama],[],"[Ryan Gosling, David Morse, Clea DuVall]",[Alex Smith]


In [28]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])

In [29]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew
3571,173,"20,000 Leagues Under the Sea","[A, ship, sent, to, investigate, a, wave, of, ...","[Adventure, Drama, ScienceFiction]","[diving, ocean, submarine, julesverne, captain...","[KirkDouglas, JamesMason, PaulLukas]",[RichardFleischer]
4091,20862,Smoke Signals,"[Young, Indian, man, Thomas, is, a, nerd, in, ...","[Drama, Comedy]",[],"[AdamBeach, EvanAdams, IreneBedard]",[ChrisEyre]
545,9741,Unbreakable,"[An, ordinary, man, makes, an, extraordinary, ...","[ScienceFiction, Thriller, Drama]","[fathersonrelationship, trainaccident, comicbo...","[BruceWillis, SamuelL.Jackson, RobinWright]",[M.NightShyamalan]
4753,139998,Supporting Characters,"[Two, New, York, film, editors, balance, their...","[Drama, Comedy]","[marriageproposal, relationship, filmmaking, f...","[AlexKarpovsky, TarikLowe, ArielleKebbel]",[DanielSchechter]
463,161795,Déjà Vu,"[L.A., shop, owner, Dana, and, Englishman, Sea...","[Romance, Drama]","[love, american, pin, stranger, ruby]","[StephenDillane, VictoriaFoyt, VanessaRedgrave]",[HenryJaglom]


In [30]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [31]:
movies.sample(5)

,movie_id,title,overview,genres,keywords,cast,crew,tags
4395,20296,Chocolate: Deep Dark Secrets,"[Christmas, Eve,, London., While, the, snow-cl...",[Thriller],[],"[AnilKapoor, SunilShetty, ArshadWarsi]",[VivekAgnihotri],"[Christmas, Eve,, London., While, the, snow-cl..."
1343,2163,Breakdown,"[When, his, SUV, breaks, down, on, a, remote, ...","[Drama, Action, Thriller]","[california, bank, ransom, carbreakdown, kidna...","[KurtRussell, J.T.Walsh, KathleenQuinlan]",[JonathanMostow],"[When, his, SUV, breaks, down, on, a, remote, ..."
2863,6020,Sorority Boys,"[Three, friends, who, head, the, Social, Commi...",[Comedy],"[sorority, crossdressing, university, tshirt, ...","[BarryWatson, MichaelRosenbaum, HarlandWilliams]",[WallaceWolodarsky],"[Three, friends, who, head, the, Social, Commi..."
1496,10719,Elf,"[When, young, Buddy, falls, into, Santa's, gif...","[Comedy, Family, Fantasy]","[holiday, elves, santaclaus, lookingforbirthpa...","[WillFerrell, JamesCaan, ZooeyDeschanel]",[JonFavreau],"[When, young, Buddy, falls, into, Santa's, gif..."
1163,10184,He's Just Not That Into You,"[Remember, that, really, cute, guy, who, said,...","[Comedy, Romance, Drama]","[relationship, ensemblecast, duringcreditsstin...","[BenAffleck, JenniferAniston, DrewBarrymore]",[KenKwapis],"[Remember, that, really, cute, guy, who, said,..."


In [32]:
movies = movies[['movie_id', 'title', 'tags']]

In [33]:
movies

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."
...,...,...,...
4804,9367,El Mariachi,"[El, Mariachi, just, wants, to, play, his, gui..."
4805,72766,Newlyweds,"[A, newlywed, couple's, honeymoon, is, upended..."
4806,231617,"Signed, Sealed, Delivered","[""Signed,, Sealed,, Delivered"", introduces, a,..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [34]:
movies['tags'] = movies['tags'].apply(lambda x: " ".join(x))

In [35]:
movies.sample(5)

,movie_id,title,tags
2280,581,Dances with Wolves,"Wounded Civil War soldier, John Dunbar tries t..."
1194,11362,The Count of Monte Cristo,Edmond Dantés's life and plans to marry the be...
2153,8848,The Bank Job,Terry is a small-time car dealer trying to lea...
1178,187,Sin City,Welcome to Sin City. This town beckons to the ...
1040,113464,Inchon,A noisy and absurd re-telling of the great 195...


In [36]:
movies['tags'] = movies['tags'].apply(lambda x: x.lower())

In [37]:
movies.sample(5)

,movie_id,title,tags
1824,208763,Forbidden Kingdom,early 18th century. cartographer jonathan gree...
1262,11025,New York Minute,top student jane ryan heads to manhattan for a...
40,49013,Cars 2,star race car lightning mcqueen and his pal ma...
2290,385383,Fight Valley,"when tory coro turns up dead, the neighborhood..."
448,2289,Cold Mountain,in this classic story of love and devotion set...


## Applying Vectorization and Cosine Similarity

In [38]:
ps = PorterStemmer()

In [39]:
def stem_words(text):
    
    y = []
    
    for i in text.split():
        
        y.append(ps.stem(i))
        
    return " ".join(y) 

In [40]:
movies['tags'] = movies['tags'].apply(stem_words)

In [41]:
movies.sample(5)

,movie_id,title,tags
1483,202575,Thunder and the House of Magic,"thunder, an abandon young cat seek shelter fro..."
39,20526,TRON: Legacy,"sam flynn, the tech-savvi and dare son of kevi..."
3191,1808,Velvet Goldmine,almost a decad ha elaps sinc bowi esqu glam su...
3696,10822,Savage Grace,thi examin of a famou scandal from the 1970 ex...
831,9618,Tango & Cash,ray tango and gabriel cash are narcot detect w...


In [42]:
vectorizer = CountVectorizer(max_features=5000, stop_words='english')

In [43]:
vector_of_tags = vectorizer.fit_transform(movies['tags']).toarray()

In [44]:
similarity_matrix = cosine_similarity(vector_of_tags)

## Recommending top 5 movies for a given movie name

In [45]:
def recommend_top_5_similar_movies_for_the_given_movie_input(movie_name):
    
    index_of_the_movie = movies[movies['title'] == movie_name].index[0]
    
    all_distances = similarity_matrix[index_of_the_movie]
    
    top_five_movies = sorted(list(enumerate(all_distances)), reverse=True, key=lambda x:x[1])[1:6] 

    for i in top_five_movies:
        
        print(movies.iloc[i[0]].title)
    

In [46]:
recommend_top_5_similar_movies_for_the_given_movie_input('Avatar')

Aliens vs Predator: Requiem
Aliens
Falcon Rising
Independence Day
Titan A.E.


## Creating a pickle file out of the cleaned dataset and the similarity matrix

In [52]:
pickle.dump(movies.to_dict(), open('cleaned_movies_dataset_dictionary.pkl', 'wb'))

In [54]:
pickle.dump(similarity_matrix, open('similarity_matrix.pkl', 'wb'))